# SDSS Redhift Distribution

This notebook plots a histagram of redshifts for the SDSS SN Sample.

In [ ]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sndata.sdss import sako18

pd.set_option('display.max_columns', 999)

# Output directory for figures
fig_dir = Path('./notebook_figs')
fig_dir.mkdir(exist_ok=True, parents=True)


We load in a table listing all available targets in the SDSS SNe survey and note the available catagories.


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sdss_df = sako18.load_table('master').to_pandas(index='CID')
    
list(sdss_df.Classification.unique())


We bin the number of targets by redshift for each catagory.

In [ ]:
sdss_df['z'] = sdss_df.zCMB
missing_spec_z = sdss_df.zCMB < 0
sdss_df.loc[missing_spec_z, 'z'] = sdss_df[missing_spec_z].zPSNIDzspec

bins = np.arange(0, 1, .05)
groups = sdss_df.groupby(['Classification', pd.cut(sdss_df.z, bins)])
binned_data = groups.size().unstack()
binned_data.fillna(0, inplace=True)
binned_data


In [ ]:
unknown = binned_data.loc['Unknown']
snia_spec = binned_data.loc[['SNIa', 'SNIa?']].sum()
snia_hostspec = binned_data.loc['zSNIa']
snia_phot = binned_data.loc['pSNIa']

snia = snia_spec + snia_hostspec + snia_phot
snibc = binned_data.loc[['SNIb', 'SNIc', 'zSNIbc', 'pSNIbc']].sum() 


Finally, we can plot the redshift distribution.

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(6, 6))

# The combined distribution
axis.hist(bins[:-1], len(bins) - 1, weights=unknown + snia, color='C0')

# Shade in the spectroscopic distribution
axis.hist(bins[:-1], len(bins) - 1, 
          weights=unknown + snia_spec + snia_hostspec, 
          histtype='step', 
          fill=False, 
          hatch='///',
          color='C3', 
          label=f'SNe with Host z ({int(snia_hostspec.sum())})')

# Shade in the direct spectroscopic distribution
axis.hist(bins[:-1], len(bins) - 1, weights=unknown + snia_spec, color='C0')
axis.hist(bins[:-1], len(bins) - 1, 
          weights=unknown + snia_spec, 
          histtype='step', 
          fill=False, 
          hatch='\\\\\\', 
          color='C1', 
          label=f'SNe with Spectral z ({int(snia_spec.sum())})')

# Shade in the unclassified distribution
axis.hist(bins[:-1], len(bins) - 1, weights=unknown, color='C0')
axis.hist(bins[:-1], len(bins) - 1, 
          weights=unknown, 
          histtype='step', 
          fill=False, 
          hatch='xx',
          color='k',
          label=f'Unknown Classification ({int(unknown.sum())})')

# Replot the combined distribution so we have clean borders
# and a legend entry with the desired symbol
axis.hist(bins[:-1], len(bins) - 1, 
          weights=unknown + snia, 
          color='C0', 
          histtype='step', 
          fill=False, 
          label=f'SNe with Photometric z ({int(snia_phot.sum())})')


# Plot lines at key redshift values
# z = (obsv_lambda / rest_lambda) - 1
u_eff = 3550
g_eff = 4680
r_eff = 6160
z_ug = (g_eff / u_eff) - 1
z_ur = (r_eff / u_eff) - 1
axis.axvline(z_ug, color='black', linestyle='--', label=r'u $\rightarrow$ g')
axis.axvline(z_ur, color='black', linestyle='-.', label=r'u $\rightarrow$ r')
    
axis.set_xlabel('Redshift', fontsize=12)
axis.set_ylabel('Number of Targets (SNe + Unclassified)', fontsize=12)
plt.xlim(0, 1)
plt.tight_layout()

# Reorder the legend
handles, labels = axis.get_legend_handles_labels()
order = [5, 2, 3, 4, 0, 1]
axis.legend(np.array(handles)[order], np.array(labels)[order], loc='upper right', framealpha=1)

plt.savefig(fig_dir / 'sdss_redshift.pdf')
plt.show()
